In [1]:
from codebase.classes import Particles
from codebase.classes_data import Data
import pandas as pd
import numpy as np
from codebase.file_utils import (
    save_obj,
    load_obj,
    make_folder,
    path_backslash
)
from codebase.ibis import essl, exp_and_normalise, model_phonebook
from tqdm import tqdm
from scipy.special import logsumexp




## Big 5 AZ

In [2]:
existing_directory = None
task_handle = 'big5_az'
gen_model = 0

if existing_directory is None:
    log_dir = make_folder(task_handle)  
    print("\n\nCreating new directory: %s" % log_dir)

else:
    log_dir = existing_directory
    log_dir = path_backslash(log_dir)
    print("\n\nReading from existing directory: %s" % log_dir)




Creating new directory: ./log/20201207_153841_big5_az/


In [3]:

# generate data
exp_data = Data(
    name = task_handle, 
    model_num = 'big5', 
    size = 50,
    random_seed = 0
    )
    
exp_data.generate()





Reading data for women


N = 677, J= 15, K =5


In [5]:

model_num = 2
## setup particles
param_names = model_phonebook(model_num)['param_names']
latent_names = model_phonebook(model_num)['latent_names']
particles = Particles(
    name = 'normal',
    model_num = model_num,
    size = 1000,
    param_names = param_names,
    latent_names = latent_names)
particles.set_log_dir(log_dir)
if gen_model:
    particles.compile_prior_model()
    particles.compile_model()
else:
    particles.load_prior_model()
    particles.load_model()

particles.sample_prior_particles(exp_data.get_stan_data()) # sample prior particles
particles.reset_weights() # set weights to 0
log_lklhds = np.empty(exp_data.size)
degeneracy_limit = 0.5
for t in tqdm(range(exp_data.size)):
    particles.get_incremental_weights(
        exp_data.get_stan_data_at_t(t)
        )
    log_lklhds[t] =  particles.get_loglikelihood_estimate()
    particles.update_weights()
    
    if (essl(particles.weights) < degeneracy_limit * particles.size) and (t+1) < exp_data.size:
        particles.resample_particles()
        particles.jitter(exp_data.get_stan_data_upto_t(t+1))
        particles.reset_weights()
    else:
        particles.update_weights()

save_obj(log_lklhds, 'log_lklhds', log_dir)


100%|██████████| 50/50 [5:00:33<00:00, 360.68s/it]  


In [6]:

print('\n\n')
marg_lklhd = np.exp(logsumexp(log_lklhds))
print('Marginal Likelihood %.5f'%marg_lklhd)

for name in ['alpha', 'Marg_cov']:
    samples = np.squeeze(particles.particles[name])
    w = exp_and_normalise(particles.weights)
    print('\n\nEstimate')
    print(np.round(np.average(samples,axis=0, weights=w),2))




Marginal Likelihood 0.00000


Estimate
[ 0.03 -0.17  0.08 -0.03  0.19  0.23 -0.02 -0.   -0.06  0.06 -0.22 -0.2
  0.19  0.07  0.15]


Estimate
[[ 0.57  0.19  0.13  0.12  0.07  0.06  0.07  0.03 -0.02  0.    0.07 -0.08
   0.11  0.13  0.09]
 [ 0.19  1.17  0.35  0.26  0.12  0.27  0.29  0.15  0.01  0.04  0.1  -0.29
   0.3   0.29  0.3 ]
 [ 0.13  0.35  0.77  0.22  0.1   0.39  0.21  0.11 -0.1   0.04  0.01 -0.26
   0.17  0.1   0.19]
 [ 0.12  0.26  0.22  1.04  0.1   0.26  0.16  0.08 -0.02  0.04  0.04 -0.22
   0.3   0.22  0.24]
 [ 0.07  0.12  0.1   0.1   0.99  0.09  0.09  0.07 -0.    0.01  0.03 -0.09
   0.1   0.08  0.09]
 [ 0.06  0.27  0.39  0.26  0.09  0.86  0.18  0.09 -0.1   0.05 -0.08 -0.27
   0.19  0.06  0.19]
 [ 0.07  0.29  0.21  0.16  0.09  0.18  0.87  0.4   0.2   0.02 -0.01 -0.25
   0.17  0.12  0.23]
 [ 0.03  0.15  0.11  0.08  0.07  0.09  0.4   0.97  0.35 -0.03 -0.06 -0.21
   0.06 -0.01  0.15]
 [-0.02  0.01 -0.1  -0.02 -0.   -0.1   0.2   0.35  1.18 -0.05 -0.05 -0.05
   0.04  0.    0.08]


In [7]:
marg_lklhd

1.634929751556244e-06